In [2]:
import PIL
from PIL import Image
import requests
from io import BytesIO
import webcolors
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import misc, ndimage
from PIL import ImageEnhance



In [3]:
'''# renommer les images à executer une seule fois !
#changez ça en fonction de vos répertoires
i=1
for f in listdir("./images"):
    if isfile(join("./images", f)) : 
        os.rename(join("./images", f), join("./images",str(i)+".jpg"))
        i+=1
'''

'# renommer les images à executer une seule fois !\n#changez ça en fonction de vos répertoires\ni=1\nfor f in listdir("./images"):\n    if isfile(join("./images", f)) : \n        os.rename(join("./images", f), join("./images",str(i)+".jpg"))\n        i+=1\n'

In [4]:
#fonction RGB ==> color_name
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS21_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]


In [5]:
#useless pour le moment 
def paper_exp(image) : 
    detected_colors = []
    white_parts= 0
    image = image.resize((300, 300))
    top_left = image.crop((0, 0, 150, 150))
    top_right = image.crop((150, 0, 300, 150))
    bottom_left = image.crop((0, 150, 150, 300))
    bottom_right = image.crop((150, 150, 300, 300))
    '''for part in [top_left , top_right , bottom_left , bottom_right]:
        for x in range(part.width) :
            for y in range(part.height) : 
                detected_colors.append(closest_colour(image.getpixel((x, y))))'''
    for part in [top_left, top_right, bottom_left, bottom_right]:
        if ((str(top_colos(part, 1).index[0])) == 'white' and (top_colos(part, 1))[0] > 0.99 ):
            white_parts += 1
    return white_parts         
    


In [6]:
#retourne le nbr de contour dans un dessin exploitée comme étant le nombre de traits qui forme le dessin
def contouring(image): 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 30, 200)
    contours, hierarchy = cv2.findContours(edged,
                                           cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return len(contours)


In [7]:
#retourne n couleurs dominantes
def top_colors(image, n):
    # convert the image to rgb
    image = image.convert('RGB')

    # resize the image to 300 x 300
    image = image.resize((300, 300))

    detected_colors = []
    for x in range(image.width):
        for y in range(image.height):
            detected_colors.append(closest_colour(image.getpixel((x, y))))
    Series_Colors = pd.Series(detected_colors)
    output = Series_Colors.value_counts()/len(Series_Colors)
    return(output.head(n))




In [8]:
#retourne le pourcentage de surface exploitée du support ainsi que la position de cette dernière par rapport à x et y
def exp_area(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(
        blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    x, y, w, h = cv2.boundingRect(thresh)
    return round((w*h)/(image.shape[0]*image.shape[1])*100, 2), round((x)/(image.shape[0])*100, 2), round((y)/(image.shape[1])*100, 2)
    


In [13]:
#initialisation 
list_outputs = []
#list_white_parts = []
list_contours = []
list_exp_areas=[]
list_positions=[]

#récupère la listes de noms de fichier
list_images = [f for f in listdir("./images") if isfile(join("./images", f))]


In [14]:
#extraction des couleurs
for image in list_images:
    print("traitement sur : "+image)
    img = Image.open(join('./images/'+image))
    list_outputs.append(list(top_colors(img, 5).index))
    img = cv2.imread(join('./images/'+image))
    list_contours.append(contouring(img))
    area, x, y = exp_area(img)
    list_exp_areas.append(area)
    list_positions.append([x,y])


traitement sur : 1.jpg
traitement sur : 1.png
traitement sur : 2.jpg
traitement sur : 3.jpg
traitement sur : 5.png
traitement sur : 6.png
traitement sur : 9.png


In [23]:
dict_images = {}
'''dict_images.update({"Image": list_images, "color": list_outputs,
                   "white_part": list_white_parts, "contour": list_contours,"exp_area":list_exp_areas , 'position' : list_positions})
'''
dict_images.update({"Image": list_images, "color": list_outputs, 'position': list_positions, "nbr_contour": list_contours,
                   "exp_area": list_exp_areas, "density": [i / j for i, j in zip(list_contours, list_exp_areas)]})


In [21]:
df = pd.DataFrame(dict_images)
df

,Image,color,nbr_contour,exp_area,density,position
0,1.jpg,"[white, silver, gray, yellow, teal]",510,89.26,5.713646,"[0.0, 7.25]"
1,1.png,"[white, silver, green, olive, lime]",26,90.56,0.287102,"[0.0, 6.38]"
2,2.jpg,"[white, silver, gray, yellow, teal]",1374,95.77,14.346873,"[0.0, 2.38]"
3,3.jpg,"[white, silver, gray, orange, yellow]",1263,96.92,13.031366,"[0.0, 2.25]"
4,5.png,"[silver, gray, olive, orange, maroon]",0,86.49,0.000000,"[5.19, 7.0]"
5,6.png,"[black, gray, white, silver, teal]",65,100.00,0.650000,"[0.0, 0.0]"
6,9.png,"[gray, silver, white, black, teal]",125,100.00,1.250000,"[0.0, 0.0]"


# NOTES 
- je dois regrouper les couleurs 
